In [105]:
import pandas as pd

In [106]:
finalTraining = pd.read_csv("/Users/sisteven/Desktop/Sports Pricing Project/Final_Cleaned_Dataset/finalTraining.csv")
final2023 = pd.read_csv("/Users/sisteven/Desktop/Sports Pricing Project/Final_Cleaned_Dataset/final2023.csv")

In [107]:
all_data = pd.concat([finalTraining, final2023], axis=0).reset_index(drop=True)

In [108]:
#Create markers for further identification
all_data['Split'] = all_data['Season'].apply(lambda x: 'Test' if x == 2023 else 'Train')

In [109]:
all_data["Date"] = pd.to_datetime(all_data["Date"])

In [110]:
#Create a new date column
all_data["date"] = all_data["Date"].dt.day

In [111]:
#Since Day and effective_day are mostly the same, and since all difference between day and effective day is either MON-SUN or SUN-SAT (in both cases a six_day delay),
# I decided to delete both column and add a new binary column called delayed (1 or 0)

all_data["delay"] = (all_data["effective_day"] != all_data["Day"]).astype(int)

In [112]:
#Drop these columns
all_data = all_data.drop(["Season","Date", "event_name"], axis = 1)

In [113]:
#Note: last_homestand was planning to be dropped in the future anyway
all_data = all_data.drop(["last_homestand"], axis = 1)

In [114]:
#One Hot Encoding
one_hot_encoded_all = pd.get_dummies(all_data, columns=["O_Team", "Month", "effective_day", "Day"])

In [115]:
from sklearn.impute import SimpleImputer

In [116]:
#Missing value imputation
columns_to_impute = ['playoff_odds', 'weighted_odds']

# Create a SimpleImputer instance with the 'mean' strategy
imputer = SimpleImputer(strategy='mean')

# Fit the imputer to the selected columns and transform them
one_hot_encoded_all[columns_to_impute] = imputer.fit_transform(one_hot_encoded_all[columns_to_impute])

In [117]:
#Normalization (Selectively)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

excluded_columns = ["Tickets", "Split"]

columns_to_scale = [col for col in one_hot_encoded_all.columns if col not in excluded_columns]

scaled_columns = scaler.fit_transform(one_hot_encoded_all[columns_to_scale])

one_hot_encoded_all[columns_to_scale] = scaled_columns

In [118]:
one_hot_encoded_all = one_hot_encoded_all.drop(["effective_day_discrepancy", "delay"], axis = 1)

In [119]:
one_hot_encoded_all_FinalTrain = one_hot_encoded_all[one_hot_encoded_all["Split"] == "Train"]
one_hot_encoded_all_FinalTest = one_hot_encoded_all[one_hot_encoded_all["Split"] == "Test"]

In [120]:
X = one_hot_encoded_all_FinalTrain.drop(["Tickets", "Split"], axis = 1)
y = one_hot_encoded_all_FinalTrain["Tickets"]

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### LASSO Regression

In [122]:
import numpy as np
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.linear_model import Lasso

In [123]:
#Lasso
lasso = Lasso(max_iter=10000, random_state=42)

parameters = {'alpha': np.logspace(-4, -2, 20)}

lasso_model = GridSearchCV(estimator=lasso, param_grid=parameters, scoring='neg_mean_absolute_error', cv=10, verbose=1)

lasso_model.fit(X_train, Y_train)

print('Best alpha:', lasso_model.best_params_['alpha'])

print('Best score (neg_mean_absolute_error):', lasso_model.best_score_)

y_pred_lasso = lasso_model.predict(X_test)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


/Users/sisteven/opt/anaconda3/envs/pynell/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.573e+07, tolerance: 1.345e+06
  model = cd_fast.enet_coordinate_descent(
/Users/sisteven/opt/anaconda3/envs/pynell/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.588e+07, tolerance: 1.401e+06
  model = cd_fast.enet_coordinate_descent(
/Users/sisteven/opt/anaconda3/envs/pynell/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of 

/Users/sisteven/opt/anaconda3/envs/pynell/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.635e+08, tolerance: 1.283e+06
  model = cd_fast.enet_coordinate_descent(
/Users/sisteven/opt/anaconda3/envs/pynell/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.267e+08, tolerance: 1.363e+06
  model = cd_fast.enet_coordinate_descent(
/Users/sisteven/opt/anaconda3/envs/pynell/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of 

Best alpha: 0.01
Best score (neg_mean_absolute_error): -3725.901459921561


/Users/sisteven/opt/anaconda3/envs/pynell/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.111e+08, tolerance: 1.512e+06
  model = cd_fast.enet_coordinate_descent(


In [124]:
# Test Performance
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test, y_pred_lasso)

print("Mean Absolute Error (Lasso):", mae)

Mean Absolute Error (Lasso): 4537.279992985688


### -----------------
### GradientBoost

In [125]:
from sklearn.ensemble import GradientBoostingRegressor
model_GB = GradientBoostingRegressor()

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
}

cv_GB = GridSearchCV(model_GB, param_grid, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1)

cv_GB.fit(X_train, Y_train)

print("Best hyperparameters:", cv_GB.best_params_)

best_model_GB = cv_GB.best_estimator_

y_pred_GB = best_model_GB.predict(X_test)
mae = mean_absolute_error(Y_test, y_pred_GB)
print(f"Mean Absolute Error: {mae}")

Best hyperparameters: {'learning_rate': 0.2, 'n_estimators': 100}
Mean Absolute Error: 3640.874983727155


### Gradient Boosting

In [126]:
import xgboost as xgb

# Initialize an XGBoost regressor object.
xgbr = xgb.XGBRegressor(objective='reg:squarederror')

# Define the parameter grid to search over.
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
}

model_xgb = GridSearchCV(estimator=xgbr, param_grid=param_grid, cv=10, scoring='neg_mean_absolute_error', verbose=1)

model_xgb.fit(X_train, Y_train)

print("Best parameters found: ", model_xgb.best_params_)

best_model_XGB = model_xgb.best_estimator_

predictions_XGB = best_model_XGB.predict(X_test)

mae = mean_absolute_error(Y_test, predictions_XGB)
print(f"Mean Absolute Error: {mae}")


Fitting 10 folds for each of 48 candidates, totalling 480 fits
Best parameters found:  {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200, 'subsample': 0.8}
Mean Absolute Error: 3748.452582465278


In [127]:
from xgboost import plot_importance
import matplotlib.pyplot as plt

### Ensemble Learning

In [128]:
y_pred_final = (predictions_XGB + y_pred_GB)/2

In [129]:
final = mean_absolute_error(Y_test, y_pred_final)

print("Final:", final)

Final: 3647.789602997971


In [130]:
Testing = pd.DataFrame()
Testing["True"] = Y_test
Testing["LASSO"] = y_pred_lasso
Testing["XGB"] = predictions_XGB
Testing["GB"] = y_pred_GB

pd.set_option("display.max_rows", None)
Testing

,True,LASSO,XGB,GB
46,40333.0,44816.485529,40879.144531,41132.480849
9,26782.0,23626.195914,28942.019531,29915.673269
206,40514.0,39770.965783,39397.902344,40201.247970
139,36570.0,38538.536887,39228.398438,36611.136120
60,36519.0,46568.491991,37435.078125,38700.047859
33,37715.0,32600.378235,34722.726562,34453.856055
297,39378.0,31606.766985,31607.017578,30921.009900
186,29274.0,27614.863642,30747.865234,28389.400437
296,34308.0,30283.945276,30315.529297,29423.217473
277,41404.0,40152.827883,38802.597656,40078.361054


### Final Prediction

In [ ]:
one_hot_encoded_all_FinalTest

In [134]:
X_Test_Prediction = one_hot_encoded_all_FinalTest.drop(["Tickets", "Split"], axis = 1)

In [135]:
#Gradient Boosting
y_pred_GB_Final = best_model_GB.predict(X_Test_Prediction)

In [136]:
#XGBoost
predictions_XGB_Final = best_model_XGB.predict(X_Test_Prediction)

In [137]:
#Ensemble
Final_Prediction = (y_pred_GB_Final + predictions_XGB_Final)/2

In [138]:
Submission = pd.read_csv("/Users/sisteven/Desktop/Sports Pricing Project/Prediction 2023 Data/2023_Ticketing_Data.csv")

In [139]:
Submission["tickets"] = Final_Prediction

In [140]:
Submission = Submission.drop(["Tickets", "Date"], axis = 1)

In [142]:
Submission["name"] = "Group 6"

In [149]:
Submission = Submission.reset_index(drop = True)

In [152]:
Submission.to_csv("submission.csv", index = False)